# Regressão Linear Múltipla: Trabalhando com Planilhas (1)
Vamos considerar, agora, um exemplo de regressão que utiliza uma base de dados CSV com valores *categóricos*. Lembre-se: da maneira como vimos até agora, precisamos trabalhar com dados numéricos. Vamos fazer, primeiro, uma conversão desses valores para, depois, explorar o algoritmo.

Vamos considerar o dataset `50_Startups.csv`, que contém uma lista de 50 empresas (startups). Essa base possui cinco colunas, trazendo as seguintes informações sobre cada empresa: gastos administrativos, gastos com marketing, investimentos em P&D, a unidade federativa (estado) que a empresa está sediada e seus lucros.

Vamos construir um modelo de aprendizado de máqujna que seja capaz de prever o quão lucrativo é uma empresa a partir das informações disponíveis.

## Importando bibliotecas e a base de dados
Vamos começar importar as principais bibliotecas de ciência de dados: `pandas`, `numpy` e `matplotlib`. Em seguida, vamos importar o conjunto de dados usando a função `read_csv()`, do `pandas`.



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
dataset = pd.read_csv('./50_Startups.csv')
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


> Lembre-se que, no Google Colab, estamos carregando o arquivo no diretório temporário chamado `sample_data`. Poderíamos, alternativamente, importar os dados de outro site (como o Kaggle) ou do seu Google Drive.

As colunas são *R&D Spend*, *Administration*, *Marketing Spend*, *State* e *Profit*, represetnam os dados do nosso problema, ou seja: Investimento em P&D, Gastos Aadministrativos, Gastos com Marketing, Estado (UF) e Lucros, respectivamente.

Vamos, agora, verificar mais algumas informações sobre o conjunto de dados  usando a função `info()` do `pandas`:

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


Como dito anteriormente, temos cinco colunas, sendo quatro numéricas e uma com valor categórico (no caso, o nome do estado - *State*). Teremos que transformar esse valor categórico em valores numérico. Isso é feito ainda no ainda na etapa de **pré-processamento**. Antes, no entanto, vamos fazer a separação das variáveis dependentes e independentes.

Como nosso objetivo é prever o lucro, esse valor será a nossa variável dependente. Vamos, portanto, deixá-lo "isolado" na variável $y$, enquanto mantemos as demais variávies (*atributos*) na matriz $X$.

In [5]:
X = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

# Codificando os Dados Categóricos
A coluna *State*, como observado, é composta de dados categóricos. Para lidar com esses dados, precisamos transformá-los em valores numéricos. Para isso, vamos utilizar a classo `OneHotEncoder`, da biblioteca `sckit-learn`. O resultado dessa transformação recebe o nome de variável *dummy*.

Variáveis *dummy* são atributos categóricos que podem assumir dois valores: 0 ou 1, indicando, respectivamente, a ausência ou presença desse atributo. No nosso caso, a coluna *State* assume apenas três valores distintos, ou seja, três estados: *California*, *New York* e *Florida*. Para transformar essa coluna em uma variável *dummy*, precisamos convertê-la em **três novas colunas**, cada uma representando um estado.

Por exemplo, consider o seguinte fragmento da base:
```
    State
0   California
1   Florida
2   New York
3   Florida
```
Após a transformação, teremos algo assim:
```
   California  Florida  New York
0   1           0         0
1   0           1         0
2   0           0         1
3   0           1         0
```
Observe que, quando há presença do Estado naquela linha, a coluna do respectivo Estado assume o valor `1`. Quando não há, assume o valor `0`. Na prática, poderíamos até remover uma dessas colunas pois essa transformações insere certa redundância na base. Por exemplo, observe a terceira linha: sabemos que aquela startup não está sediada na Califórnia e nem na Flórida; logo, ela estará em Nova York, ou seja, poderíamos remover essa última coluna sem perder uma informação relevante.

Vamos, portanto, codificar a codificar a coluna *State*. Para isso, importar e usar dois métodos da biblioteca `scikit-learn`: `ColumnTransformer`, presente na classe `composer` e `OneHotEncoder`, da classe `preprocessing`.

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Invoca o método ColumnTransformer. O primeiro parâmetro indica que será
# codificada a coluna 3 usando o método OneHotEncoder. O segundo parâmetro
# indica que as colunas serão mantidas inalteradas.
ct = ColumnTransformer(transformers =[('encoder', OneHotEncoder(), [3])],
                       remainder ='passthrough')

# Aplica a transformação na matriz X
X = np.array(ct.fit_transform(X))

In [7]:
X

array([[0.0, 0.0, 1.0, 165349.2, 136897.8, 471784.1],
       [1.0, 0.0, 0.0, 162597.7, 151377.59, 443898.53],
       [0.0, 1.0, 0.0, 153441.51, 101145.55, 407934.54],
       [0.0, 0.0, 1.0, 144372.41, 118671.85, 383199.62],
       [0.0, 1.0, 0.0, 142107.34, 91391.77, 366168.42],
       [0.0, 0.0, 1.0, 131876.9, 99814.71, 362861.36],
       [1.0, 0.0, 0.0, 134615.46, 147198.87, 127716.82],
       [0.0, 1.0, 0.0, 130298.13, 145530.06, 323876.68],
       [0.0, 0.0, 1.0, 120542.52, 148718.95, 311613.29],
       [1.0, 0.0, 0.0, 123334.88, 108679.17, 304981.62],
       [0.0, 1.0, 0.0, 101913.08, 110594.11, 229160.95],
       [1.0, 0.0, 0.0, 100671.96, 91790.61, 249744.55],
       [0.0, 1.0, 0.0, 93863.75, 127320.38, 249839.44],
       [1.0, 0.0, 0.0, 91992.39, 135495.07, 252664.93],
       [0.0, 1.0, 0.0, 119943.24, 156547.42, 256512.92],
       [0.0, 0.0, 1.0, 114523.61, 122616.84, 261776.23],
       [1.0, 0.0, 0.0, 78013.11, 121597.55, 264346.06],
       [0.0, 0.0, 1.0, 94657.16, 145077.58

Observe que, como explicado antes, a coluna *State* foi transformada outras três colunas, agora com valores binários. A única diferença é que as colunas foram movidas para o início da *array*.

## Dividindo o dataset em "Treinamento" e "Teste"
Como visto na aula anterior, vamos dividir o conjunto de dados em dois conjuntos: treinamento e teste. Manteremos a proporção 80/20, ou seja, 80% dos dados para treinamento e 20% para validação e teste. Utilizaremos, novamente, a biblioteca `scikit-learn`.



In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size=0.2)

## Treinando e Validando o Modelo
Finalmente, vamos efetivamente implementar o nosso modelo de regressão linear múltipla. O treinamente será feito com os dados presentes em `X_train`, que é uma matriz composta pelas variáveis dependentes, e em `y_train`, composta pelas variáveis dependentes.

Note que não há diferenças entre o que será implementado aqui e o que foi implementado para o modelo de regressão linear simples. A biblioteca do `sklearn` oferece a mesma classe (`LinearRegression`) como solução.

In [9]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

Uma vez treinado o modelo, vamos verificar como ele se comporta com o conjunto de dados de teste. Para isso, utilizaremos a variável `X_test` e `y_test`, além do método `predict`, também presente na classe `LinearRegression`.

In [10]:
y_pred = regressor.predict(X_test)

print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test),1)), axis=1))

[[103015.20159795 103282.38      ]
 [132582.27760816 144259.4       ]
 [132447.73845175 146121.95      ]
 [ 71976.09851258  77798.83      ]
 [178537.48221057 191050.39      ]
 [116161.24230167 105008.31      ]
 [ 67851.69209676  81229.06      ]
 [ 98791.73374687  97483.56      ]
 [113969.43533014 110352.25      ]
 [167921.06569552 166187.94      ]]


Obviamente, não podemos simplesmente confiar em uma análise visual. Por esse motivo, devemos recorrer a técnicas bem estabelecidas, como o $R^2$ ou o menor erro quadrático. Vamos avaliar ambas aqui:

In [11]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
r2 = r2_score(y_test, y_pred)

Note que usamos uma outra função para computar o valor do coeficiente de determinação (nosso `r2`). O resultado será o mesmo; esse exemplo apenas ilustra outras abordagens.

In [12]:
r2

0.9347068473282546

In [13]:
rmse

9137.9901527941

Lembre-se apenas que: quanto maior o valor de $R^2$, melhor a será a precisão. Já o erro quadrático médio (ou a raiz quadrada desse erro) é uma métrica a ser minimizada.